In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import plotly as py
pd.__version__

'1.1.5'

In [2]:
# all files stored in ~/Input folder
SSF = pd.read_csv("Input/Subsidies_SSF_split_2019_corrected.csv")
# original SSF and LSF split subsidies all subtypes all countries
countries = pd.read_csv("Input/Country_Data.csv")
# countries
catch_all = pd.read_csv("Input/map/LVCatch_16_byFEEEZ_v2_24Oct20.csv")
# original data dump from Vicky Lam (sea around us) for 2016 catch and landing data
sector = pd.read_csv("Input/Fishing_sector.csv")
# Sector_type_id code
reporting_status  = pd.read_csv("Input/Reporting_status.csv")
# reporting_status_id code
catch_type = pd.read_csv("Input/catch_type.csv")
# EEZ id based on SAU
EEZ_ID = pd.read_csv("Input/EEZ_ID.csv")
# all country subsidies and quality scores 
# read in file created in 'Subsidies_2018_Confidence'
subsidies_quality_scores = pd.read_csv("Input/subsidies_quality_scores.csv")
# read in EEZ quality score data
EEZ_quality_scores =  pd.read_csv("Input/catch_rel_score_eez_2021-7-28.csv")

In [3]:
# rename columns
catch_all = catch_all.rename(columns = {'CountryName':'Country'})
# merge fishing sector info
# fishing sector id: 1= industrial, 2=subsistence, 3 = artisanal, 4 = recreational
catch_sector = pd.merge(left = catch_all, right = sector, how = 'left')
# merge reporting status info
# reporting_status_id 1 = reported; 2 = unreported
catch_report = pd.merge(left = catch_sector, right = reporting_status, how = 'left')
# merge catch info
# Catch type status 1 = landing ; 2 = discard
catch = pd.merge(left = catch_report, right = catch_type , how = 'left')
# check numbers of countries
catch.Country.nunique()

199

In [4]:
# prepare country name replacement dictionary so EEZ country names are the same as subsidies country names
replacement = dict(zip(['Korea (North)','Korea (South)',
                                                     'Antigua & Barbuda',
                                                     'Bosnia & Herzegovina',
                                                     'Brunei Darussalam',
                                                     'Congo, R. of',
                                                     'Congo (ex-Zaire)',
                                                     "C�te d'Ivoire",
                                                     'South Cyprus',
                                                     'Dominican Republic',
                                                     'Marshall Isl.',
                                                     'Sao Tome & Principe',
                                                     'Solomon Isl.',
                                                     'Saint Kitts & Nevis',
                                                     'Saint Lucia',
                                                     'Saint Vincent & the Grenadines',
                                                     'Syrian Arab Republic',
                                                     'Trinidad & Tobago',
                                                     'United Arab Emirates'],['Korea D P Rp',
                                                                       'Korea Rep', 
                                                                      'Antigua Barb', 
                                                                      'Bosnia Herzg', 
                                                                      'Brunei Darsm',
                                                                      'Congo Republic',
                                                                      'Congo Dem Rep',
                                                                      "Cote d'Ivoire",
                                                                      'Cyprus',
                                                                      'Dominican Rp',
                                                                      'Marshall Is',
                                                                      'Sao Tome Prn',
                                                                      'Solomon Is',
                                                                      'St Kitts Nev',
                                                                      'St Lucia',
                                                                      'St Vincent',
                                                                      'Syria',
                                                                      'Trinidad Tob',
                                                                      'Untd Arab Em']))

In [5]:
# apply the replacement for country names to catch dataframe
catch['Country'] = catch['Country'].replace(replacement)
# as some countries registered in the SAU database with a fishing fleet which 
# were not taken into account in the original subsidies database these countries will be deleted
# make a list of all countries from original country df
countries_keep = countries['Country'].unique().tolist()
# delete all rows based on Countries (Fishing Fleet not EEZ) that are not in the subsidies database
catch_clean = catch[catch['Country'].isin(countries_keep)]
# check numbers of unique country names
print(catch_clean.Country.nunique())
countries.Country.nunique()

152


153

In [6]:
# Monaco has a subsidy allocated but not a fishing fleet in SAU database, Monaco will be deleted
list_1 = catch_clean['Country'].unique().tolist()
missing_countries = np.setdiff1d(countries_keep,list_1)
missing_countries

array(['Monaco'], dtype='<U18')

In [7]:
# all na in EEZName column refer to high seas i.e. need to be replaced
# catch_clean[['EEZName']] = catch_clean[['EEZName']].fillna('high_seas')
catch_clean.loc[:, 'EEZName'].fillna('high_seas', inplace=True)

/home/anna/.pyenv/versions/3.6.7/envs/DWF/lib/python3.6/site-packages/pandas/core/series.py:4536: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [8]:
# only keep essential columns in the dataframe
catch_clean1 = catch_clean.loc[:,['Country', 'eezid', 'EEZName','scientific_name', 'common_name','fishing_sector',
       'reporting_status', 'catch_type', 'sumcatch','LandVal']]
catch_clean1.Country.nunique()

152

In [9]:
# remove all discards from catch_clean1 column name: catch_type_id and value = 'Discards'
catch_clean2 = catch_clean1[catch_clean1['catch_type'] != 'discard']
catch_clean2.Country.nunique()

152

In [10]:
# remove Recreational, Artisinal and Subsistence catches to only keep Industrial 
catch_clean3 = catch_clean2[catch_clean2['fishing_sector'] == 'industrial']
catch_clean3.Country.nunique()

139

In [11]:
# sum all Landed VALUES by country to then compute the percentage of total landed value in each EEZ
value_group =catch_clean3.groupby(['Country','EEZName'], as_index = False).sum()
value_group_country = catch_clean3.groupby(['Country'], as_index = False).sum()
value_country = value_group_country[['Country','LandVal']]
value_country = value_country.rename(columns = {'LandVal':'total_landval'})
# merge the total landed value (USD) per country info to actual landed value per EEZ and calculate proportion 
value_country_EEZ = pd.merge(left = value_group, right = value_country, how = 'left')
# calculate proportion of total landed value per fleet (country) to each landed value coming from different EEZs
value_country_EEZ['landval_percent'] = value_country_EEZ['LandVal']/value_country_EEZ['total_landval']
value_country_EEZ.Country.nunique()

139

In [12]:
# prepare subsidies data to merge with catch data to split all capacity-enhancing subsidies by EEZ
SSF_cap = SSF[SSF['Category'] == 'Capacity-enhancing']
# group by country as subsidy types are not relevant
SSF_cap_group = SSF_cap.groupby(['Country'], as_index = False).sum()
# compute LSF by subtracting SSF subsidies from total subsidies
# only ned LSF for now as they will split into EEZ catches/landed value as assume all SSF subsidies are
# only for domestic waters and not in DWF or High seas by definition. 
SSF_cap_group['LSF_USD'] = SSF_cap_group['Constant 2018 USD']-SSF_cap_group['SSF_USD']
# Group by Country and LSF subsidies
LSF_subsidies = SSF_cap_group.loc[:, ('Country', 'LSF_USD')]
print (LSF_subsidies['LSF_USD'].sum()/1000000000)
print (SSF_cap_group['Constant 2018 USD'].sum()/1000000000)

18.333122492285494
22.246636513341365


In [13]:
LSF_subsidies.Country.nunique()

152

In [14]:
# add subsidies data to VALUE data by merging based on Country name as unique identifyer
LSF_value_subsidies = pd.merge(left = value_country_EEZ, right = LSF_subsidies, how = 'left')
# calculate subsidies proportion based on Landed Value proportion within EEZ from each Country
LSF_value_subsidies['Subsidies_valueprop_USD'] = LSF_value_subsidies['landval_percent']*LSF_value_subsidies['LSF_USD']
# take slice from df with only relevant columns so SSF subsidies can be added (append)
Subsidies_EEZ_LSF_value = LSF_value_subsidies.loc[:,('Country','EEZName','Subsidies_valueprop_USD')]
Subsidies_EEZ_LSF_value = Subsidies_EEZ_LSF_value.rename(columns = {'Subsidies_valueprop_USD':'Subsidies_EEZ_value'})
print(Subsidies_EEZ_LSF_value.Subsidies_EEZ_value.sum()/1000000000)

18.333122492285494


In [15]:
# prepare SSF subsidies dataframe to append to the Subsidies_EEZ which only includes LSF subsidies
SSF_EEZ = SSF_cap_group.loc[:,('Country','SSF_USD')]
# will create a new column that names each countries EEZ into 'Country EEZ' so it can be appended to LSF subsidies
SSF_EEZ['EEZName'] = SSF_EEZ['Country'] + ' SSF'

In [16]:
# prepare to match subsidies data divided into EEZ based on landed value
SSF_EEZ_value = SSF_EEZ.rename(columns = {'SSF_USD':'Subsidies_EEZ_value'})
# change column order to match Subsidies_EEZ_LSF column order
SSF_EEZ_value = SSF_EEZ_value[['Country','EEZName','Subsidies_EEZ_value']]


In [17]:
# append SSF_EEZ_value to Subsidies_EEZ_LSF_value
Subsidies_EEZ_value = Subsidies_EEZ_LSF_value.append(SSF_EEZ_value)
Subsidies_EEZ_value.sort_values(by = ['Country'], inplace = True)
# Subsidies_EEZ_value.to_csv("Output/Subsidies_EEZ_for_map.csv")
Subsidies_EEZ_value

,Country,EEZName,Subsidies_EEZ_value
0,Albania,Albania,4.429411e+06
0,Albania,Albania SSF,2.558283e+06
1,Algeria,Algeria SSF,3.620263e+07
1,Algeria,Algeria,1.722205e+08
2,Angola,Angola,2.328702e+07
...,...,...,...
150,Viet Nam,Viet Nam SSF,1.835094e+08
1020,Yemen,Somalia,7.596584e+07
1019,Yemen,Djibouti,8.722758e+06
1021,Yemen,Yemen (Arabian Sea),1.516235e+07


In [18]:
replacement_EEZ = dict(zip(['Korea (North)',
                            'Korea (South)',
                            'American Samoa',
                            'Antigua & Barbuda',
                            'Bosnia & Herzegovina',
                            'Brunei Darussalam',
                            'Comoros Isl.',
                            'Congo, R. of',
                            'Congo (ex-Zaire)',
                            "C™te d'Ivoire","Côte d'Ivoire",
                            'South Cyprus',
                            'Dominican Republic',
                            'Marshall Isl.',
                            "Micronesia (Federated States of)",
                            'Sao Tome & Principe',
                            'Solomon Isl.',
                            'Saint Kitts & Nevis',
                            'Saint Lucia',
                            'Saint Vincent & the Grenadines',
                            'Syrian Arab Republic',
                            'Trinidad & Tobago',
                            'United Arab Emirates',
                            'Galapagos Isl. (Ecuador)',
                            'Hawaii Main Islands (USA)',
                            'Hawaii Northwest Islands (USA)',
                            'Norfolk Isl. (Australia)',
                            'Heard & McDonald Isl. (Australia)',
                            'Lord Howe Isl. (Australia)',
                            'Macquarie Isl. (Australia)',
                            'Trindade & Martim Vaz Isl. (Brazil)',
                            'Juan Fernandez Islands (Chile)',
                            'Desventuradas Isl. (Chile)',
                            'Crozet Isl. (France)',
                            'Corsica (France)',
                            'St Paul & Amsterdam Isl. (France)',
                            'Kerguelen Isl. (France)',
                            'Crete (Greece)',
                            'Andaman & Nicobar Isl. (India)',
                            'Sardinia (Italy)',
                            'Sicily (Italy)',
                            'Kermadec Isl. (New Zealand)',
                            'Svalbard Isl. (Norway)',
                            'Jan Mayen Isl. (Norway)',
                            'Azores Isl. (Portugal)', 
                            'Madeira Isl. (Portugal)',
                            'Prince Edward Isl. (South Africa)',
                            'Balearic Island (Spain)',
                            'Canary Isl. (Spain)',
                            'Palmyra Atoll & Kingman Reef (USA)',
                            'Northern Marianas (USA)',
                            'Wake Isl. (USA)','Guam (USA)',
                            'Howland & Baker Isl. (USA)',
                            'Jarvis Isl. (USA)',
                            'Johnston Atoll (USA)', 
                            'Channel Isl. (UK)'],['Korea D P Rp',
                                                      'Korea Rep', 
                                                      'Samoa',
                                                      'Antigua Barb', 
                                                      'Bosnia Herzg', 
                                                      'Brunei Darsm',
                                                      'Comoros',
                                                      'Congo Republic',
                                                      'Congo Dem Rep',
                                                      "Cote d'Ivoire","Cote d'Ivoire",
                                                      'Cyprus',
                                                      'Dominican Rp',
                                                      'Marshall Is',
                                                      'Micronesia',
                                                      'Sao Tome Prn',
                                                      'Solomon Is',
                                                      'St Kitts Nev',
                                                      'St Lucia',
                                                      'St Vincent',
                                                      'Syria',
                                                      'Trinidad Tob',
                                                      'Untd Arab Em',
                                                      'Ecuador (Galalagos Isl.)',
                                                      'USA (Hawaii Main Islands)',
                                                      'USA (Hawaii Northwest Islands)',
                                                  'Norfolk Isl. (Australia)',
                                                  'Heard & McDonald Isl. (Australia)',
                                                  'Australia (Lord Howe Isl.)',
                                                  'Australia (Macquarie Isl.)',
                                                  'Brazil (Trindade & Martim Vaz Isl.)',
                                                  'Chile (Juan Fernandez Islands)',
                                                  'Chile (Desventuradas Isl.)',
                                                  'Crozet Isl. (France)',
                                                  'France (Corsica)',
                                                  'St Paul & Amsterdam Isl. (France)',
                                                  'Kerguelen Isl. (France)',
                                                  'Greece (Crete)',
                                                  'India (Andaman & Nicobar Isl.)',
                                                  'Italy (Sardinia)',
                                                  'Italy (Sicily)',
                                                  'New Zealand (Kermadec Isl.)',
                                                  'Svalbard Isl. (Norway)',
                                                  'Jan Mayen Isl. (Norway)',
                                                  'Portugal (Azores Isl.)',
                                                  'Portugal (Madeira Isl.)',
                                                  'South Africa (Prince Edward Isl.)',
                                                  'Spain (Balearic Island)',
                                                  'Spain (Canary Isl.)',
                                                  'Palmyra Atoll & Kingman Reef (USA)',
                                                  'Northern Marianas (USA)',
                                                  'Wake Isl. (USA)',
                                                  'Guam (USA)',
                                                  'USA (Howland & Baker Isl.)',
                                                  'Jarvis Isl. (USA)',
                                                  'Johnston Atoll (USA)', 
                                                  'United Kingdom (Channel Isl.)']))

In [19]:
# apply the name replacement of Countries in EEZ column to match Country names in Country columns
Subsidies_EEZ_value['EEZName'] = Subsidies_EEZ_value['EEZName'].replace(replacement_EEZ)
# check using Samoa example
Subsidies_EEZ_value[Subsidies_EEZ_value['EEZName'].str.contains('Samoa')]

,Country,EEZName,Subsidies_EEZ_value
386,Japan,Samoa,1.515182e+06
439,Korea Rep,Samoa,7.287528e+05
740,Samoa,Samoa,1.122965e+05
113,Samoa,Samoa SSF,5.053273e+05
953,USA,Samoa,8.246535e+04
929,USA,Samoa,8.670429e+04


In [20]:
# create a new column that contains the actual country name for each EEZ to merge all 
# EEZs for each country together

def EEZ_merge(row):
     if row['EEZName'].startswith(row['Country']):
          val = row['Country']

     elif row['Country'] not in row['EEZName']:
          val = row['EEZName']

     else:
          val = row['EEZName']
     return val


Subsidies_EEZ_value['EEZ_merged'] = Subsidies_EEZ_value.apply(EEZ_merge, axis=1)
# check dataframe using New Zealand as example
Subsidies_EEZ_value[Subsidies_EEZ_value['EEZName'].str.contains('New Zealan')]

,Country,EEZName,Subsidies_EEZ_value,EEZ_merged
23,Australia,Tokelau (New Zealand),1.020630e+01,Tokelau (New Zealand)
188,Dominica,New Zealand,2.193228e+06,New Zealand
411,Japan,New Zealand,3.901504e+06,New Zealand
463,Korea Rep,New Zealand,1.418479e+07,New Zealand
455,Korea Rep,New Zealand (Kermadec Isl.),4.165576e+05,New Zealand (Kermadec Isl.)
604,New Zealand,New Zealand (Kermadec Isl.),2.037864e+03,New Zealand
606,New Zealand,New Zealand,2.762300e+07,New Zealand
97,New Zealand,New Zealand SSF,0.000000e+00,New Zealand
948,USA,Niue (New Zealand),3.900491e+03,Niue (New Zealand)
955,USA,Tokelau (New Zealand),2.809147e+06,Tokelau (New Zealand)


In [21]:
# some oversea territories will be merged into the subsidies provider and receiver 
# by their "mother-country" e.g. Falkand Island is already part of UK source and now also will become part 
# of the UK sink, this means the Falkland Island EEZ will be merged with the UK EEZ 

def sink_country(row):
     
    if 'Netherlands' in row['EEZ_merged']:
        return 'Netherlands'
    if 'Australia' in row['EEZ_merged']:
        return 'Australia'
    if 'Norway' in row['EEZ_merged']:
        return 'Norway'
    if 'Brazil' in row['EEZ_merged']:
        return 'Brazil'
    if 'Canada' in row['EEZ_merged']:
        return 'Canada'
    if 'UK' in row['EEZ_merged']:
        return 'United Kingdom'
    if 'United Kingdom' in row['EEZ_merged']:
        return 'United Kingdom'
    if 'France' in row['EEZ_merged']:
        return 'France'
    if 'Colombia' in row['EEZ_merged']:
        return 'Colombia'
    if 'Denmark' in row['EEZ_merged']:
        return 'Denmark'
    if 'Chile' in row['EEZ_merged']:
        return 'Chile'
    if 'Ecuador' in row['EEZ_merged']:
        return 'Ecuador'
    if 'Germany' in row['EEZ_merged']:
        return 'Germany'
    if 'USA' in row['EEZ_merged']:
        return 'USA'
    if 'India' in row['EEZ_merged']:
        return 'India'
    if 'Indonesia' in row['EEZ_merged']:
        return 'Indonesia'
    if 'Iran' in row['EEZ_merged']:
        return 'Iran'
    if 'Japan' in row['EEZ_merged']:
        return 'Japan'
    if 'Kiribati' in row['EEZ_merged']:
        return 'Kiribati'
    if 'Korea (South)' in row['EEZ_merged']:
        return 'Korea Rep'
    if 'Malaysia' in row['EEZ_merged']:
        return 'Malaysia'
    if 'Morocco' in row['EEZ_merged']:
        return 'Morocco'
    if 'New Zealand' in row['EEZ_merged']:
        return 'New Zealand'
    if 'Portugal' in row['EEZ_merged']:
        return 'Portugal'
    if 'Russia' in row['EEZ_merged']:
        return 'Russian Federation'
    if 'South Africa' in row['EEZ_merged']:
        return 'South Africa'
    if 'Spain' in row['EEZ_merged']:
        return 'Spain'
    if 'Sweden' in row['EEZ_merged']:
        return 'Sweden'
    if 'Turkey' in row['EEZ_merged']:
        return 'Turkey'
    if 'Yemen' in row['EEZ_merged']:
        return 'Yemen'
    if 'Micronesia' in row['EEZ_merged']:
        return 'Micronesia'
    if 'Greenland' in row['EEZ_merged']:
        return 'Denmark'
    
    
    return row['EEZ_merged']

Subsidies_EEZ_value['EEZ_country_sink'] = Subsidies_EEZ_value.apply(sink_country, axis=1)

In [22]:
# This function categorizes the source and sink countries into 3 categories:
# domestic: where a country fishes in their own EEZ (including those of their oversea territories)
# distant_water: countries fish in other country's EEZ
# high_seas: waters outside of any EEZs

def EEZ_category(row):
    
    if 'high_seas' in row['EEZ_country_sink']:
        return 'high_seas'
    
    if row['EEZ_country_sink'].startswith(row['Country']):
        return 'domestic'

    #if row['Country'] not in row['EEZ_merged']:
        #return 'distant_water'
    
    return 'distant_water'


Subsidies_EEZ_value['EEZ_category'] = Subsidies_EEZ_value.apply(EEZ_category, axis=1)
# Subsidies_EEZ_value.to_csv("Output/Subsidies_EEZ_value.csv")

In [23]:
# read in subsidies quality score per country and merge with subsidies and EEZ df
subsidies_quality = subsidies_quality_scores[['Country','quality_score']]
subsidies_quality.columns = ['Country','sub_quality_score']
subsidies_quality.sub_quality_score.unique()

array([1, 4, 3, 2])

In [24]:
# merge subsidies quality score based on country name to Subsidies_EEZ_value
Subsidies_EEZ_value_qual = pd.merge(left = Subsidies_EEZ_value, right = subsidies_quality, how = 'left')
Subsidies_EEZ_value_qual.sub_quality_score.unique()
# check for which countries no sub_quality_scores showed up
Subsidies_EEZ_value_qual[Subsidies_EEZ_value_qual.isna().any(axis=1)]
# Find list of countries that have NaN in sub_quality_score

# drop the rows as they all have a Subsidies_EEZ_value of zero??? 
Subsidies_EEZ_value_qual[Subsidies_EEZ_value_qual['EEZName'].str.contains('Albania')]

,Country,EEZName,Subsidies_EEZ_value,EEZ_merged,EEZ_country_sink,EEZ_category,sub_quality_score
0,Albania,Albania,4.429411e+06,Albania,Albania,domestic,1.0
1,Albania,Albania SSF,2.558283e+06,Albania,Albania,domestic,1.0


In [25]:
# prepare EEZ quality score data to  merge to subsidies EEZ df
EEZ_quality_scores_2011 = EEZ_quality_scores[EEZ_quality_scores['period'] == '2011 onwards']
# EEZ_quality_scores_2011.columns
LSF_EEZ_quality_scores = EEZ_quality_scores_2011[EEZ_quality_scores_2011['sector'] == 'Industrial']
LSF_EEZ_quality_scores_dom = LSF_EEZ_quality_scores[LSF_EEZ_quality_scores['layer'] == 'Reconstructed domestic catch']
LSF_EEZ_quality_scores_dom = LSF_EEZ_quality_scores_dom[['eez','score']]
LSF_EEZ_quality_scores_dom.columns = ['EEZName', 'EEZ_quality_score']
# check if all scores have been merged correclty
print(LSF_EEZ_quality_scores_dom.EEZ_quality_score.unique())
LSF_EEZ_quality_scores_dom[LSF_EEZ_quality_scores_dom['EEZName'].str.contains('Albania')]

[2 0 1 3]


,EEZName,EEZ_quality_score
3,Albania,2


In [26]:
# merge EEZ quality scores with main subsidies dataframe
Subsidies_EEZ_quality = pd.merge(left = Subsidies_EEZ_value_qual, right = LSF_EEZ_quality_scores_dom, how = 'left')
print(Subsidies_EEZ_quality.EEZ_quality_score.unique())

# write a function to replace all nan in Subsidies_EEZ_quality['EEZ_quality_score'] with 1.0 
# (as all high seas and distant water catches in SAU have a score 1 across all EEZs) but
# only where EEZ_category is not domestic. 
missing_mask = Subsidies_EEZ_quality['EEZ_quality_score'].isna()
mapping_dict = dict({'high_seas': 1.0, 'distant_water': 1.0})
Subsidies_EEZ_quality.loc[missing_mask, 'EEZ_quality_score']  = Subsidies_EEZ_quality.loc[
    missing_mask, 'EEZ_category'].map(mapping_dict)


[ 2. nan  1.  0.  3.]


In [27]:
# change quality scores into percentages following SAU methods Methods-Catch&Allocation-www.seaaroundus.org
# table 1
# 4.0 == +/- 10%
# 3.0 == +/- 20%
# 2.0 == +/- 30%
# 1.0 == +/- 50%
scores_di = dict({4.0: 0.10, 3.0: 0.20, 2.0:0.30, 1.0:0.50})
Subsidies_EEZ_CI = Subsidies_EEZ_quality.replace({"EEZ_quality_score": scores_di, "sub_quality_score": scores_di})
# check dataframe
Subsidies_EEZ_CI

,Country,EEZName,Subsidies_EEZ_value,EEZ_merged,EEZ_country_sink,EEZ_category,sub_quality_score,EEZ_quality_score
0,Albania,Albania,4.429411e+06,Albania,Albania,domestic,0.5,0.3
1,Albania,Albania SSF,2.558283e+06,Albania,Albania,domestic,0.5,NaN
2,Algeria,Algeria SSF,3.620263e+07,Algeria,Algeria,domestic,0.1,NaN
3,Algeria,Algeria,1.722205e+08,Algeria,Algeria,domestic,0.1,0.3
4,Angola,Angola,2.328702e+07,Angola,Angola,domestic,0.2,0.5
...,...,...,...,...,...,...,...,...
1169,Viet Nam,Viet Nam SSF,1.835094e+08,Viet Nam,Viet Nam,domestic,0.3,NaN
1170,Yemen,Somalia,7.596584e+07,Somalia,Somalia,distant_water,0.5,0.5
1171,Yemen,Djibouti,8.722758e+06,Djibouti,Djibouti,distant_water,0.5,0.0
1172,Yemen,Yemen (Arabian Sea),1.516235e+07,Yemen,Yemen,domestic,0.5,0.3


In [28]:
# calculate compound quality score by taking average of subsidies and EEZ quality scores
Subsidies_EEZ_CI['compound_quality_score'] = (Subsidies_EEZ_CI['sub_quality_score']+
                                                   Subsidies_EEZ_CI['EEZ_quality_score'])/ 2

In [29]:
# left nan's are from SSF subsidies, so fill compound scores with values from subsidies quality score for all SSF
Subsidies_EEZ_CI.compound_quality_score.fillna(Subsidies_EEZ_CI.sub_quality_score, inplace=True)
Subsidies_EEZ_CI_dropna = Subsidies_EEZ_CI.dropna(subset=['compound_quality_score'])
Subsidies_EEZ_CI_dropna['compound_quality_score'].unique()

array([0.4 , 0.5 , 0.1 , 0.2 , 0.35, 0.25, 0.3 , 0.15, 0.15, 0.05])

In [30]:
# check total subsidies amound (USD) to make sure no rows/info was lost anywhere
Subsidies_EEZ_CI_dropna.Subsidies_EEZ_value.sum()

22246636513.341362

In [31]:
Subsidies_EEZ_CI_dropna['lower_bound_subsidies_USD'] = Subsidies_EEZ_CI_dropna[
    'Subsidies_EEZ_value']-(Subsidies_EEZ_CI_dropna['Subsidies_EEZ_value']*Subsidies_EEZ_CI_dropna[
    'compound_quality_score'])
Subsidies_EEZ_CI_dropna['upper_bound_subsidies_USD'] = Subsidies_EEZ_CI_dropna[
    'Subsidies_EEZ_value']+(Subsidies_EEZ_CI_dropna['Subsidies_EEZ_value']*Subsidies_EEZ_CI_dropna[
    'compound_quality_score'])
Subsidies_EEZ_CI_dropna.columns

/home/anna/.pyenv/versions/3.6.7/envs/DWF/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/home/anna/.pyenv/versions/3.6.7/envs/DWF/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Index(['Country', 'EEZName', 'Subsidies_EEZ_value', 'EEZ_merged',
       'EEZ_country_sink', 'EEZ_category', 'sub_quality_score',
       'EEZ_quality_score', 'compound_quality_score',
       'lower_bound_subsidies_USD', 'upper_bound_subsidies_USD'],
      dtype='object')

In [32]:
# shorten dataframe to only essential columns
Subsidies_EEZ_CI_all = Subsidies_EEZ_CI_dropna[['Country', 'EEZName', 'Subsidies_EEZ_value', 'EEZ_merged',
       'EEZ_country_sink', 'EEZ_category','lower_bound_subsidies_USD', 'upper_bound_subsidies_USD']]

In [33]:
# group all data into Source and Sink (impact) countries
Subsidies_sink_source_country =  Subsidies_EEZ_CI_dropna.groupby(
    ['Country','EEZ_country_sink'], as_index = False).sum()

Subsidies_sink_source_country_short = Subsidies_sink_source_country[[
    'Country', 'EEZ_country_sink', 'Subsidies_EEZ_value',
    'lower_bound_subsidies_USD', 'upper_bound_subsidies_USD']]
Subsidies_sink_source_country_short.sum()

Subsidies_sink_source_country_short = Subsidies_sink_source_country_short.rename(
    columns={"EEZ_country_sink": "Sink_country"})
# Subsidies_sink_source_country_short.to_csv("Output/Source_sink_countries_CI.csv")

In [34]:
Subsidies_EEZ_categories_global = Subsidies_EEZ_CI_all.groupby(['EEZ_category'], as_index = False).sum()
# Subsidies_EEZ_categories_global.to_csv("Output/Global_categories_CI.csv")
Subsidies_EEZ_categories_global

,EEZ_category,Subsidies_EEZ_value,lower_bound_subsidies_USD,upper_bound_subsidies_USD
0,distant_water,5.968098e+09,4.366077e+09,7.570119e+09
1,domestic,1.489270e+10,1.087349e+10,1.891190e+10
2,high_seas,1.385843e+09,8.231161e+08,1.948570e+09


In [35]:
Subsidies_EEZ_categories = Subsidies_EEZ_CI_all.groupby(['Country','EEZ_category'], as_index = False).sum()
# Subsidies_EEZ_categories.to_csv("Output/Country_EEZcategory_CI.csv")
Subsidies_EEZ_categories

,Country,EEZ_category,Subsidies_EEZ_value,lower_bound_subsidies_USD,upper_bound_subsidies_USD
0,Albania,domestic,6.987694e+06,3.936788e+06,1.003860e+07
1,Algeria,domestic,2.084232e+08,1.703588e+08,2.464875e+08
2,Angola,distant_water,2.465297e+05,2.218767e+05,2.711827e+05
3,Angola,domestic,4.280351e+07,3.074976e+07,5.485727e+07
4,Angola,high_seas,6.552985e+00,4.259440e+00,8.846530e+00
...,...,...,...,...,...
327,Viet Nam,distant_water,1.572342e+04,9.434049e+03,2.201278e+04
328,Viet Nam,domestic,3.342166e+08,2.188809e+08,4.495523e+08
329,Viet Nam,high_seas,3.621235e+06,2.172741e+06,5.069729e+06
330,Yemen,distant_water,8.468860e+07,4.452499e+07,1.248522e+08
